# 4 SQLite и работа с базой данных

#### 4.1 Открытие соединения

Для подключения к базе данных используется функция `connect()`

In [19]:
import sqlite3
conn = sqlite3.connect('example.db')

Файл бд будет автоматически создан 

In [20]:
!ls

010-iterables.ipynb  040-sqlite.ipynb  impex	    testmod.py
020-arch-json.ipynb  config.json       __pycache__  Untitled.ipynb
030-imports.ipynb    example.db        _static


Если были произведены операции записи или создания таблиц, то надо сперва сделать `comit()` а потом `close()`

In [22]:
conn.commit()
conn.close()

ProgrammingError: Cannot operate on a closed database.

In [23]:
!rm example.db

Иногда Можно так же не создавать файл базы данных, а использовать базу данных прямо в памяти. Для этого используется ключевое слово ` :memory: `

In [24]:
import sqlite3
mem_db = sqlite3.connect(':memory:')

Как мы видим файл не создался

In [25]:
!ls

010-iterables.ipynb  040-sqlite.ipynb  __pycache__  Untitled.ipynb
020-arch-json.ipynb  config.json       _static
030-imports.ipynb    impex	       testmod.py


Но соединение всё равно желательно закрыть

In [26]:
mem_db.commit()
mem_db.close()

### 4.2 Создание таблиц

In [27]:
import sqlite3
mem_db = sqlite3.connect(':memory:')
curr = mem_db.cursor()
curr.execute('''
CREATE TABLE 
    test_table(id INTEGER PRIMARY KEY, descr TEXT, qty REAL)
''')

Добавление записей в таблицы

In [28]:
curr.execute("INSERT INTO test_table VALUES (1,'some description text', 35.14)")



In [29]:
curr.execute("INSERT INTO test_table VALUES (1,'some description text', 35.14)")

IntegrityError: UNIQUE constraint failed: test_table.id

In [30]:
curr.execute("INSERT INTO test_table VALUES (10,'some description text', 35.14)")
curr.execute("INSERT INTO test_table VALUES (15,'some description text', 35.14)")


In [31]:
mem_db.commit()

In [32]:
curr = mem_db.cursor()
curr.execute('SELECT ROWID, * FROM test_table')
for row in curr:
    print(row)

(1, 1, 'some description text', 35.14)
(10, 10, 'some description text', 35.14)
(15, 15, 'some description text', 35.14)


Просмотр всех таблиц в базе данных

In [33]:
import sqlite3
conn = sqlite3.connect(':memory:')
curr = conn.cursor()
curr.execute('''
CREATE TABLE 
    test_table(id INTEGER PRIMARY KEY, descr text, qty real)
''')
curr.execute('''
CREATE TABLE 
    test_table1(id INTEGER PRIMARY KEY, descr text, qty real)
''')

conn.commit()
curr.execute("SELECT name from sqlite_master WHERE type='table'")

tables = list()
for table in curr:
    print(table)
    tables.append(table)
    
for table in tables:
    curr.execute("SELECT * FROM {}".format(table[0]))
    print(curr.fetchall())
conn.close()

('test_table',)
('test_table1',)
[]
[]


### 4.3 Задача

![](_static/cmpl-db.png)

1. Создание таблицы `control` со структурой (идентификатор комплаенса, описание комплаенса).
2. Чтение json файла `controls.json` и загрузка данных из него в таблицу `control`
3. Создание таблицы `scandata` С результатми сканирования, структура (идентификатор комплаенса, описание комплаенса, статус)  

Пример json файла

In [18]:
[
  ["000","File named 'testfile' is present in the root folder on target system."],
  ["456","some other control"]
]

[['123', 'some control'], ['456', 'some other control']]

![](_static/cmpl-add-control.png)

Реализовать функцию `add_control(id_, status)`: которая будет добавлять данные в таблицу scandata. 
Статусы контролей:
    
* STATUS_COMPLIANT = 1 
* STATUS_NOT_COMPLIANT = 2
* STATUS_NOT_APPLICABLE = 3
* STATUS_ERROR = 4
* STATUS_EXCEPTION = 5

Фукнция должна брать описание контроля из таблицы `control` по его идентификатору.

Написать первый тестовый скрипт `000_test_file_exists.py`. Скрипт будет проверять наличие тестового файла testfile в корневой директории, сканируемой системы. Если файл существует то, то в таблицу scandata идёт запись со статусом STATUS_COMPLIANT иначе со статусом STATUS_NOT_COMPLIANT. В случае ошибок подключения со статусом STATUS_ERROR